In [23]:
!pip install underthesea
from underthesea import word_tokenize
import re
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer

^C


ModuleNotFoundError: No module named 'underthesea'

In [2]:
df0=pd.read_csv('scraped_data/crawling_0.csv')
df1=pd.read_csv('scraped_data/crawling_1.csv')
df2=pd.read_csv('scraped_data/crawling_2.csv')
df3=pd.read_csv('scraped_data/crawling_3.csv')
df4=pd.read_csv('scraped_data/crawling_4.csv')
df5=pd.read_csv('scraped_data/crawling_5.csv')
df6=pd.read_csv('scraped_data/crawling_6.csv')
df7=pd.read_csv('scraped_data/crawling_7.csv')
df8=pd.read_csv('scraped_data/crawling_8.csv')
df9=pd.read_csv('scraped_data/crawling_9.csv')
df10=pd.read_csv('scraped_data/crawling_10.csv')
df11=pd.read_csv('scraped_data/crawling_11.csv')
data_df=pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])

In [3]:
data_df.reset_index(drop=True, inplace=True)

In [4]:
selected_categories = data_df['class'].value_counts()[:20].index.tolist()
selected_categories

['Thời sự',
 'Thế giới',
 'Kinh doanh',
 'Thể thao',
 'Pháp luật',
 'Sức khỏe',
 'Cần biết',
 'Giáo dục',
 'Văn hóa',
 'Nhịp sống trẻ',
 'Giải trí',
 'Khoa học',
 'Du Lịch',
 'Bạn đọc làm báo',
 'Công nghệ',
 'Phóng sự',
 'Doanh nghiệp',
 'Nhà đất',
 'Giả - Thật',
 'Tuyển sinh']

In [5]:
data_df = data_df[data_df['class'].isin(selected_categories)]
data_df

,links,title,description,content,class
0,/nhu-chua-he-co-cuoc-chia-ly-gap-mat-con-mot-l...,Như chưa hề có cuộc chia ly: ‘Gặp mặt con một ...,TTO - Sau 51 năm đi khắp nơi tìm kiếm con trai...,Đây là một trong số những trường hợp trẻ em th...,Giải trí
1,/hai-nu-sinh-bi-danh-hoi-dong-tren-pho-phong-g...,"Hai nữ sinh bị đánh hội đồng trên phố, Phòng G...","TTO - Ngày 4-1, Phòng Giáo dục và đào tạo quận...",Theo thông tin xác nhận từ Phòng Giáo dục và đ...,Giáo dục
2,/diem-tin-the-thao-toi-4-1-cuu-gdkt-bong-da-nh...,Điểm tin thể thao tối 4-1: Cựu GĐKT bóng đa...,"TTO - CLB Sài Gòn có cố vấn cấp cao ""xịn"", CLB...",Cựu GĐKT bóng đá Nhật Bản làm cố vâ...,Thể thao
3,/iran-bat-tau-cho-hoa-chat-han-quoc-co-thuyen-...,Iran bắt tàu chở hóa chất Hàn Quốc có thuyền v...,TTO - Truyền thông Iran ngày 4-1 cho biết Vệ b...,Một số hãng truyền thông Iran cho biết IRGC đã...,Thế giới
4,/canh-bao-chieu-muon-danh-ngan-hang-moi-du-hoi...,Cảnh báo chiêu mượn danh ngân hàng mời dự hội ...,TTO - Giả danh ngân hàng gọi điện thoại mời mở...,"VPBank vừa phát đi cảnh báo trên vào hôm nay, ...",Kinh doanh
...,...,...,...,...,...
47995,/tham-sat-o-thai-nguyen-nghi-pham-thua-nhan-nh...,Thảm sát ở Thái Nguyên: Nghi phạm thừa nhận nh...,"TTO - Tại cơ quan công an, bước đầu nghi phạm ...",Sau khi truy bắt Hoàng Văn Chín - nghi phạm gâ...,Pháp luật
47996,/mobifone-dat-loi-nhuan-hon-6-000-ty-dong-tron...,MobiFone đạt lợi nhuận hơn 6.000 tỉ đồng trong...,TTO – Mức lợi nhuận trước thuế của MobiFone ướ...,"Ngày 26-12, MobiFone công bố kết thúc năm 2019...",Kinh doanh
47997,/chuan-bi-co-le-hoi-hoa-anh-dao-tai-tp-hcm-201...,Chuẩn bị có lễ hội hoa anh đào tại TP.HCM,TTO - Làm việc với đoàn giám sát của HĐND TP.H...,"Ngày 26-12, Đoàn công tác của HĐND TP.HCM đã c...",Du Lịch
47998,/xet-xu-vu-nu-sinh-giao-ga-bui-van-cong-lien-t...,Xét xử vụ nữ sinh giao gà: Bùi Văn Công liên t...,"TTO - Được gọi lên xét hỏi, bị cáo Bùi Văn Côn...","Chiều 26-12, phiên tòa xét xử 9 bị cáo bắt cóc...",Pháp luật


In [6]:
data_df=data_df[~((data_df['title'].isnull())|
                  (data_df['description'].isnull())|
                  (data_df['content'].isnull()))]

In [7]:
data_df.drop(['links'],axis=1,inplace=True)

C:\Users\Admin\anaconda3\envs\myenv\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
data_df.drop_duplicates(inplace=True)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
data_df

,title,description,content,class
0,Như chưa hề có cuộc chia ly: ‘Gặp mặt con một ...,TTO - Sau 51 năm đi khắp nơi tìm kiếm con trai...,Đây là một trong số những trường hợp trẻ em th...,Giải trí
1,"Hai nữ sinh bị đánh hội đồng trên phố, Phòng G...","TTO - Ngày 4-1, Phòng Giáo dục và đào tạo quận...",Theo thông tin xác nhận từ Phòng Giáo dục và đ...,Giáo dục
2,Điểm tin thể thao tối 4-1: Cựu GĐKT bóng đa...,"TTO - CLB Sài Gòn có cố vấn cấp cao ""xịn"", CLB...",Cựu GĐKT bóng đá Nhật Bản làm cố vâ...,Thể thao
3,Iran bắt tàu chở hóa chất Hàn Quốc có thuyền v...,TTO - Truyền thông Iran ngày 4-1 cho biết Vệ b...,Một số hãng truyền thông Iran cho biết IRGC đã...,Thế giới
4,Cảnh báo chiêu mượn danh ngân hàng mời dự hội ...,TTO - Giả danh ngân hàng gọi điện thoại mời mở...,"VPBank vừa phát đi cảnh báo trên vào hôm nay, ...",Kinh doanh
...,...,...,...,...
47995,Thảm sát ở Thái Nguyên: Nghi phạm thừa nhận nh...,"TTO - Tại cơ quan công an, bước đầu nghi phạm ...",Sau khi truy bắt Hoàng Văn Chín - nghi phạm gâ...,Pháp luật
47996,MobiFone đạt lợi nhuận hơn 6.000 tỉ đồng trong...,TTO – Mức lợi nhuận trước thuế của MobiFone ướ...,"Ngày 26-12, MobiFone công bố kết thúc năm 2019...",Kinh doanh
47997,Chuẩn bị có lễ hội hoa anh đào tại TP.HCM,TTO - Làm việc với đoàn giám sát của HĐND TP.H...,"Ngày 26-12, Đoàn công tác của HĐND TP.HCM đã c...",Du Lịch
47998,Xét xử vụ nữ sinh giao gà: Bùi Văn Công liên t...,"TTO - Được gọi lên xét hỏi, bị cáo Bùi Văn Côn...","Chiều 26-12, phiên tòa xét xử 9 bị cáo bắt cóc...",Pháp luật


In [10]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [11]:
data_df["content"][0]

'Đây là một trong số những trường hợp trẻ em thất lạc ở cô nhi viện Long Thành (Đồng Nai) tìm được hơi ấm tình thân ruột thịt sau mấy chục năm thất lạc nhau. Khoảng 51 năm trước, bà Võ Thị Kim Thanh gửi tạm 4 người con vào cô nhi viện Long Thành vài ngày để đi tìm chồng. Nhưng quá thương nhớ mẹ, chỉ sau 1-2 ngày sau, ông Hồ Văn Sơn bỏ cô nhi viện đi tìm mẹ. Vì còn quá nhỏ, lúc đó có 6 tuổi, ông Sơn đi lạc và được người lạ nhận nuôi. Trong khi đó, người mẹ quay lại tìm con không thấy.Cách đây vài năm, cả bà Kim Thanh và ông Sơn đều gửi thư về chương trình Như chưa hề có cuộc chia ly. Từ đó chương trình đã tìm đến xác minh thông tin và kết nối. Tuy nhiên, gần đến ngày đoàn tụ gia đình, ông Sơn mặc cảm bản thân gửi thư về chương trình xin hủy, không tìm mẹ nữa.Ông Sơn tâm sự: "Thực lòng tôi chán nản, không muốn làm gánh nặng cho người khác. Lấy vợ thì hết yêu thương, có con trai thì năm trước nó cũng bỏ đi. Có được căn nhà nhỏ thì thiếu nợ phải bỏ nhà đi bán vé số sống qua ngày.Bản thân h

In [12]:
#' '.join(gensim.utils.simple_preprocess(data_df['content'][0]))
def gensim_utils_simple_preprocess(x):
    return ' '.join(gensim.utils.simple_preprocess(x))
data_df['content']=data_df['content'].apply(gensim_utils_simple_preprocess)
data_df['class']=data_df['class'].apply(gensim_utils_simple_preprocess)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [17]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\\p{P}*)([p{L}.]*\\p{L}+)(\\p{P}*$)', r'\\1/\\2/\\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)



data_df['content']=data_df['content'].apply(chuan_hoa_dau_cau_tieng_viet)
data_df['class']=data_df['class'].apply(chuan_hoa_dau_cau_tieng_viet)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

data_df['content']=data_df['content'].apply(covert_unicode)
data_df['class']=data_df['class'].apply(covert_unicode)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


^C


ModuleNotFoundError: No module named 'underthesea'

In [16]:
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df["content"]
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)